In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

from scipy import stats

sys.path.append('..')

from src.features.build_features import df, X, X_tfidf

In [3]:
print(f"The size of the cleaned data frame is {df.shape}. The 10 first observations is shown below:")
df.head(10)

The size of the cleaned data frame is (183347, 9). The 10 first observations is shown below:


,Starttid,Sluttid,Navn,Rolle,Tekst,MeetingId,Parti,Regering,I_Regering
0,2009-10-06 12:04:01.713,2009-10-06 12:11:14.713,Lars Løkke Rasmussen,minister,Danmark er et godt land at leve i. Vi har bygg...,2A762CE9-E5C1-4C4C-A5B9-87994C68C257,V,0Lars1,True
1,2009-10-06 12:11:14.713,2009-10-06 12:14:49.713,Lars Løkke Rasmussen,minister,"Det er ikke den eneste mulighed, vi skal gribe...",2A762CE9-E5C1-4C4C-A5B9-87994C68C257,V,0Lars1,True
2,2009-10-06 12:14:49.713,2009-10-06 12:19:47.713,Lars Løkke Rasmussen,minister,For det første – og helt afgørende: Vi skal ho...,2A762CE9-E5C1-4C4C-A5B9-87994C68C257,V,0Lars1,True
3,2009-10-06 12:19:47.713,2009-10-06 12:25:11.713,Lars Løkke Rasmussen,minister,Velstand og vækst er i virkeligheden de to byg...,2A762CE9-E5C1-4C4C-A5B9-87994C68C257,V,0Lars1,True
4,2009-10-06 12:25:11.713,2009-10-06 12:29:01.713,Lars Løkke Rasmussen,minister,Regeringen arbejder på hele velfærdsområdet me...,2A762CE9-E5C1-4C4C-A5B9-87994C68C257,V,0Lars1,True
5,2009-10-06 12:29:01.713,2009-10-06 12:35:55.713,Lars Løkke Rasmussen,minister,"Hvor det er friheden, der skal have bedre livs...",2A762CE9-E5C1-4C4C-A5B9-87994C68C257,V,0Lars1,True
6,2009-10-06 12:35:55.713,2009-10-06 12:40:14.713,Lars Løkke Rasmussen,minister,"Det er jo sådan set alvorlige emner, jeg her h...",2A762CE9-E5C1-4C4C-A5B9-87994C68C257,V,0Lars1,True
7,2009-10-06 12:40:14.713,2009-10-06 12:47:11.713,Lars Løkke Rasmussen,minister,Danmark skal også være i front i kampen for fr...,2A762CE9-E5C1-4C4C-A5B9-87994C68C257,V,0Lars1,True
8,2009-10-06 12:47:11.713,2009-10-06 12:50:39.740,Lars Løkke Rasmussen,minister,"Før jeg afslutter min redegørelse, vil jeg ger...",2A762CE9-E5C1-4C4C-A5B9-87994C68C257,V,0Lars1,True
9,2009-10-07 13:04:25.897,2009-10-07 13:06:31.920,Bjarne Laustsen,medlem,"Tak, hr. formand. Den sag, jeg vil bringe på b...",4F214615-40FE-40F2-9726-D212854D78EC,S,0Lars1,False


In [4]:
pivot = pd.pivot_table(df[['Parti','Regering']],values=None, index='Regering', columns='Parti', aggfunc=len, fill_value=0)
pivot

Parti,ALT,DF,EL,KD,KF,LA,NA,RV,S,SF,UFG,V
Regering,,,,,,,,,,,,
0Lars1,0,5798,4015,143,6361,1059,190,3083,10383,5911,294,12631
Helle1,0,7856,6193,0,2988,4027,313,4603,11324,5300,15,10288
Helle2,12,3685,3661,0,1513,1931,275,2888,6899,2243,77,4799
Lars2,2351,4345,5503,0,1490,2248,572,1712,5103,2322,50,8346
Lars3,1155,2581,2328,0,1183,2216,105,880,3131,1213,28,3143


In [22]:
pivot_frac = pivot.div(pivot.sum(axis=1).values, axis = 0)
pivot_frac

Parti,ALT,DF,EL,KD,KF,LA,NA,RV,S,SF,UFG,V
Regering,,,,,,,,,,,,
0Lars1,0.000000,0.116267,0.080513,0.002868,0.127557,0.021236,0.003810,0.061823,0.208210,0.118533,0.005896,0.253289
Helle1,0.000000,0.148487,0.117054,0.000000,0.056476,0.076115,0.005916,0.087002,0.214036,0.100176,0.000284,0.194454
Helle2,0.000429,0.131687,0.130829,0.000000,0.054069,0.069006,0.009827,0.103206,0.246543,0.080156,0.002752,0.171497
Lars2,0.069062,0.127636,0.161653,0.000000,0.043769,0.066036,0.016803,0.050291,0.149903,0.068210,0.001469,0.245168
Lars3,0.064299,0.143684,0.129600,0.000000,0.065858,0.123365,0.005845,0.048990,0.174303,0.067528,0.001559,0.174971


In [25]:
print(f"On all the text stop words has been removed and stemmning has been carried out. Afterwards the dimension is {X.shape}. I.e. we have {X.shape[1]} words in the vocabulary.")

On all the text stop words has been removed and stemmning has been carried out. Afterwards the dimension is (183347, 25223). I.e. we have 25223 words in the vocabulary.
